In [36]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate,MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You take the title of a movie and reply with three emojis."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "what is three emojis of {question}"),
    ]
)


examples = [
    {
        "movie": "what is three emojis of Top Gun",
        "answer": """🛩️👨‍✈️🔥""",
    },
    {
        "movie": "what is three emojis of The Godfather",
        "answer": """👨‍👨‍👦🔫🍝""",
    }
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "what is three emojis of {movie}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You take the title of a movie and reply with three emojis."),
        example_prompt,
        ("human", "what is three emojis of {movie}?"),
    ]
)

#chain = final_prompt | chat

#chain.invoke({"movie": })


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result.content)

invoke_chain("The Lord of the Rings")


🧝‍♂️🧙‍♂️🗡️


In [37]:
invoke_chain("Forrest Gump")

🏃‍♂️🍫🏅
